In [1]:
import requests
import pandas as pd
import json

In [2]:
# Esta funcion imprime las 10 monedas volatiles con mas capitalizacion y luego las 10 monedas estables.
def get_top_coins():
    stable = []
    volatil = []

    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 100,
        "page": 1
    }

    response = requests.get(url, params=params)
    data = response.json()

    for coin in data:
        if coin['current_price'] > 1 or 0.99 > coin['current_price'] and len(volatil) < 10:
            volatil.append(coin['id'])
        elif 1.1 > coin['current_price'] > 0.97 and len(stable) < 10:
            stable.append(coin['id'])
        else:
            break
        
    return stable,volatil

stable, volatil = get_top_coins()
print(stable)
print(volatil)

['tether', 'usd-coin']
['bitcoin', 'ethereum', 'binancecoin', 'ripple', 'staked-ether', 'cardano', 'dogecoin', 'solana', 'tron', 'polkadot']


In [185]:
# Utiliza una lista de nombres de criptomonedas y a traves de la api de coingecko extrae la informacion de cada una de ellas para luego de un ETL guardar 
# la informacion en un archivo csv.
def get_historical_data(coinnamelist,days):
    for coin in coinnamelist:
        url = f"https://api.coingecko.com/api/v3/coins/{coin}/market_chart"
        params = {
            'vs_currency': 'usd',
            'days': days,
            'interval': 'daily'
        }
    
        response = requests.get(url, params=params)
    
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data)
            df['market_caps'] = df['market_caps'].apply(lambda x: int(x[1]))
            df['total_volumes'] = df['total_volumes'].apply(lambda x: int(x[1]))
            df['Date'] = df['prices'].apply(lambda x: pd.to_datetime(x[0], unit='ms'))
            df['prices'] = df['prices'].apply(lambda x: int(x[1]))
            df.rename(columns={'prices': 'Price_usd','total_volumes':'Volume','market_caps':'Market_cap'}, inplace=True)
            df = df.groupby(df["Date"].dt.to_period("M")).tail(1)
            df.to_csv(f'./Data/{coin}.csv',index=False)
        else:
            print(f"Error al obtener los datos de {coin}:", response.status_code)


In [28]:
get_historical_data(volatil)

Obtencion de datos globales

In [3]:
url = 'https://api.coingecko.com/api/v3/global'
response = requests.get(url)
if response.status_code == 200:
    data = response.json()

In [4]:
data

{'data': {'active_cryptocurrencies': 10150,
  'upcoming_icos': 0,
  'ongoing_icos': 49,
  'ended_icos': 3376,
  'markets': 836,
  'total_market_cap': {'btc': 41909168.0563866,
   'eth': 659824072.8019023,
   'ltc': 16787531520.292736,
   'bch': 5725551987.714669,
   'bnb': 5107265646.735155,
   'eos': 1869087785530.507,
   'xrp': 2084736384445.723,
   'xlm': 8719990169319.429,
   'link': 174022950370.4369,
   'dot': 247451527525.28854,
   'yfi': 196650428.46162176,
   'usd': 1111275380341.9402,
   'aed': 4081753366634.2646,
   'ars': 388900709701547.44,
   'aud': 1715246881905.506,
   'bdt': 121618100976189.36,
   'bhd': 418990824302.6044,
   'bmd': 1111275380341.9402,
   'brl': 5400909475999.873,
   'cad': 1502434312743.8823,
   'chf': 975814245304.3989,
   'clp': 958819510912830.6,
   'cny': 7986736158517.532,
   'czk': 24635820567060.652,
   'dkk': 7622962385313.375,
   'eur': 1022662281513.4741,
   'gbp': 872999047115.1628,
   'hkd': 8713454693492.135,
   'huf': 391243091509044.3,


In [5]:
data = data['data']
df_global = pd.DataFrame(data)
df_global.head()

,active_cryptocurrencies,upcoming_icos,ongoing_icos,ended_icos,markets,total_market_cap,total_volume,market_cap_percentage,market_cap_change_percentage_24h_usd,updated_at
btc,10150,0,49,3376,836,4.190917e+07,1.946119e+06,46.447861,2.646764,1692812860
eth,10150,0,49,3376,836,6.598241e+08,3.063998e+07,18.243934,2.646764,1692812860
ltc,10150,0,49,3376,836,1.678753e+10,7.795558e+08,NaN,2.646764,1692812860
bch,10150,0,49,3376,836,5.725552e+09,2.658751e+08,NaN,2.646764,1692812860
bnb,10150,0,49,3376,836,5.107266e+09,2.371640e+08,3.017118,2.646764,1692812860


In [150]:
df_global.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67 entries, btc to sol
Data columns (total 10 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   active_cryptocurrencies               67 non-null     int64  
 1   upcoming_icos                         67 non-null     int64  
 2   ongoing_icos                          67 non-null     int64  
 3   ended_icos                            67 non-null     int64  
 4   markets                               67 non-null     int64  
 5   total_market_cap                      61 non-null     float64
 6   total_volume                          61 non-null     float64
 7   market_cap_percentage                 10 non-null     float64
 8   market_cap_change_percentage_24h_usd  67 non-null     float64
 9   updated_at                            67 non-null     int64  
dtypes: float64(4), int64(6)
memory usage: 5.8+ KB


Faltan datos importante en las columnas total_market_cap y market_cap_percentage

In [6]:
df_global['market_cap_percentage'].sum()

83.74989519178826

La sumatoria de la columna market_cap_percentage (porcentaje de capitalizacion) no llega al 100 % o cercano.

In [12]:
df_global = df_global.sort_values(by='market_cap_percentage', ascending=False)
df_global.head()

AttributeError: 'list' object has no attribute 'sort_values'

***

Utilizando la opcion /coins/market de la api de coingecko para obtener datos mas completos.

In [24]:
# Esta funcion retorna los datos de todas las criptomonedas en coingecko
def all_coins():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 300,
        "page": 1
    }

    response = requests.get(url, params=params)
    data = response.json()
        
    return data

In [25]:
df_global = all_coins()

In [26]:
df = pd.DataFrame(df_global)
df

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,26494.000000,516419987700,1,5.571183e+11,1.415758e+10,26531.000000,...,2.100000e+07,2.100000e+07,69045.000000,-61.57665,2021-11-10T14:24:11.849Z,67.810000,39023.58676,2013-07-06T00:00:00.000Z,None,2023-08-23T18:27:58.125Z
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,1681.990000,202445525585,2,2.024455e+11,1.304481e+10,1687.100000,...,1.202143e+08,NaN,4878.260000,-65.47618,2021-11-10T14:24:19.604Z,0.432979,388870.94316,2015-10-20T00:00:00.000Z,"{'times': 83.87662491037823, 'currency': 'btc'...",2023-08-23T18:27:54.941Z
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.999740,82777720344,3,8.277772e+10,2.290470e+10,1.004000,...,8.284578e+10,NaN,1.320000,-24.48174,2018-07-24T00:00:00.000Z,0.572521,74.52260,2015-03-02T00:00:00.000Z,None,2023-08-23T18:25:01.017Z
3,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,217.000000,33396658632,4,4.341284e+10,7.583048e+08,218.010000,...,1.538562e+08,2.000000e+08,686.310000,-68.37213,2021-05-10T07:24:17.097Z,0.039818,545044.97998,2017-10-19T00:00:00.000Z,None,2023-08-23T18:27:59.048Z
4,ripple,xrp,XRP,https://assets.coingecko.com/coins/images/44/l...,0.529876,28119207066,5,5.314114e+10,1.030952e+09,0.533607,...,9.998850e+10,1.000000e+11,3.400000,-84.36588,2018-01-07T00:00:00.000Z,0.002686,19679.48364,2014-05-22T00:00:00.000Z,None,2023-08-23T18:28:01.128Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,nervos-network,ckb,Nervos Network,https://assets.coingecko.com/coins/images/9566...,0.002795,93396509,246,NaN,1.928300e+06,0.002816,...,NaN,NaN,0.043706,-93.58578,2021-03-31T17:08:59.155Z,0.001979,41.66407,2022-12-17T14:25:57.243Z,"{'times': -0.7205251231491501, 'currency': 'us...",2023-08-23T18:28:01.232Z
246,deso,deso,Decentralized Social,https://assets.coingecko.com/coins/images/1631...,8.820000,93032406,247,9.547245e+07,2.985653e+06,9.000000,...,1.080849e+07,1.080849e+07,198.680000,-95.56253,2021-06-19T20:53:25.052Z,5.330000,65.44907,2022-09-01T18:05:41.566Z,None,2023-08-23T18:27:50.785Z
247,cartesi,ctsi,Cartesi,https://assets.coingecko.com/coins/images/1103...,0.122722,91336069,248,1.228387e+08,4.491361e+06,0.123712,...,1.000000e+09,1.000000e+09,1.740000,-92.95776,2021-05-09T00:43:41.306Z,0.026706,359.92877,2020-05-10T04:49:27.744Z,None,2023-08-23T18:27:51.407Z
248,hxro,hxro,HXRO,https://assets.coingecko.com/coins/images/7805...,0.148912,90893638,249,1.489118e+08,5.479090e+03,0.151517,...,1.000000e+09,NaN,0.800981,-81.40881,2021-06-05T00:59:41.210Z,0.009147,1528.02516,2019-11-10T15:12:15.992Z,None,2023-08-23T18:27:46.564Z


Comprobando la esactitud calculando el porcentaje de capitalizacion o dominancia de bitcoin con los datos provistos.

In [27]:
df[df['id'] == 'bitcoin']['market_cap'][0]/df['market_cap'].sum()

0.4758397239441478

Segun https://coin360.com/ los datos son correctos.

Eliminando columnas innecesarias

In [28]:
df.drop(['image','high_24h','total_supply','max_supply','ath','ath_change_percentage','ath_date','atl','atl_change_percentage','atl_date','roi','low_24h','price_change_24h','price_change_percentage_24h','market_cap_change_24h','market_cap_change_percentage_24h','circulating_supply','fully_diluted_valuation'],axis=1,inplace=True)

In [29]:
df.head()

,id,symbol,name,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,last_updated
0,bitcoin,btc,Bitcoin,26494.000000,516419987700,1,5.571183e+11,1.415758e+10,2023-08-23T18:27:58.125Z
1,ethereum,eth,Ethereum,1681.990000,202445525585,2,2.024455e+11,1.304481e+10,2023-08-23T18:27:54.941Z
2,tether,usdt,Tether,0.999740,82777720344,3,8.277772e+10,2.290470e+10,2023-08-23T18:25:01.017Z
3,binancecoin,bnb,BNB,217.000000,33396658632,4,4.341284e+10,7.583048e+08,2023-08-23T18:27:59.048Z
4,ripple,xrp,XRP,0.529876,28119207066,5,5.314114e+10,1.030952e+09,2023-08-23T18:28:01.128Z


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       250 non-null    object 
 1   symbol                   250 non-null    object 
 2   name                     250 non-null    object 
 3   current_price            250 non-null    float64
 4   market_cap               250 non-null    int64  
 5   market_cap_rank          250 non-null    int64  
 6   fully_diluted_valuation  239 non-null    float64
 7   total_volume             250 non-null    float64
 8   last_updated             250 non-null    object 
dtypes: float64(3), int64(2), object(4)
memory usage: 17.7+ KB


El dataframe no contiene datos nulos o vacios.

***

In [45]:
def get_exchange_info():
    base_url = "https://api.coingecko.com/api/v3/exchanges"
    response = requests.get(base_url)
    
    if response.status_code == 200:
        exchanges_data = response.json()
        return exchanges_data
    else:
        print("Error:", response.status_code)
        return None

In [46]:
df_exchanges = get_exchange_info()

In [47]:
df_exchanges = pd.DataFrame(df_exchanges)
df_exchanges

,id,name,year_established,country,description,url,image,has_trading_incentive,trust_score,trust_score_rank,trade_volume_24h_btc,trade_volume_24h_btc_normalized
0,binance,Binance,2017.0,Cayman Islands,,https://www.binance.com/,https://assets.coingecko.com/markets/images/52...,False,10,1,271767.416238,178353.292417
1,gdax,Coinbase Exchange,2012.0,United States,,https://coinbase-consumer.sjv.io/coingecko,https://assets.coingecko.com/markets/images/23...,False,10,2,46180.714912,46180.714912
2,bybit_spot,Bybit,2018.0,British Virgin Islands,Bybit is a cryptocurrency exchange that offers...,https://www.bybit.com,https://assets.coingecko.com/markets/images/69...,False,10,3,36175.895076,36175.895076
3,huobi,Huobi,2013.0,Seychelles,,https://www.huobi.com,https://assets.coingecko.com/markets/images/25...,False,10,4,42230.269658,26856.506181
4,kraken,Kraken,2011.0,United States,,https://r.kraken.com/c/2223866/687155/10583,https://assets.coingecko.com/markets/images/29...,False,10,5,20650.324610,17796.345182
...,...,...,...,...,...,...,...,...,...,...,...,...
95,thena-fusion,THENA FUSION,NaN,None,,https://www.thena.fi/swap,https://assets.coingecko.com/markets/images/11...,False,6,96,291.760327,291.760327
96,pancakeswap-v3-zksync,Pancakeswap V3 (zkSync),2020.0,Singapore,,https://pancakeswap.finance/swap?chain=zkSync,https://assets.coingecko.com/markets/images/11...,False,6,97,267.204845,267.204845
97,bitazza,Bitazza,2021.0,Thailand,Bitazza is ASEAN's locally-based digital asset...,https://trade.bitazza.com/gl/exchange/,https://assets.coingecko.com/markets/images/83...,False,6,98,257.859261,257.859261
98,bitci,Bitci TR,2018.0,Turkey,,https://www.bitci.com.tr/en,https://assets.coingecko.com/markets/images/45...,False,6,99,250.923007,250.923007


In [48]:
df_exchanges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               100 non-null    object 
 1   name                             100 non-null    object 
 2   year_established                 75 non-null     float64
 3   country                          84 non-null     object 
 4   description                      98 non-null     object 
 5   url                              100 non-null    object 
 6   image                            100 non-null    object 
 7   has_trading_incentive            99 non-null     object 
 8   trust_score                      100 non-null    int64  
 9   trust_score_rank                 100 non-null    int64  
 10  trade_volume_24h_btc             100 non-null    float64
 11  trade_volume_24h_btc_normalized  100 non-null    float64
dtypes: float64(3), int64(2)

In [49]:
df_exchanges['has_trading_incentive'].unique()

array([False, None], dtype=object)

In [50]:
df_exchanges.drop(['url','image','trade_volume_24h_btc','trade_volume_24h_btc_normalized','description','has_trading_incentive'],axis=1,inplace=True)

In [51]:
df_exchanges.head()

,id,name,year_established,country,trust_score,trust_score_rank
0,binance,Binance,2017.0,Cayman Islands,10,1
1,gdax,Coinbase Exchange,2012.0,United States,10,2
2,bybit_spot,Bybit,2018.0,British Virgin Islands,10,3
3,huobi,Huobi,2013.0,Seychelles,10,4
4,kraken,Kraken,2011.0,United States,10,5
